In [1]:
import pandas as pd
import altair as alt
import numpy as np
import scipy.io
import seaborn as sns
import matplotlib.pyplot as plt
alt.data_transformers.enable('default', max_rows=None)
alt.renderers.enable('notebook')
%matplotlib inline

/miniconda3/envs/rbcenv/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
%matplotlib inline

In [3]:
import joypy
from matplotlib import cm
import seaborn as sns

In [5]:
base_dir = './'
model_id = 'pputida_ccm'
n_models = 10000

file_in_MCA = f'{base_dir}MCA_{model_id}.mat'
file_in_mets = f'{base_dir}{model_id}_metsActive.dat'
file_in_rxns = f'{base_dir}{model_id}_rxnsActive.dat'
file_in_enzs = f'{base_dir}{model_id}_enzNames.dat'

def get_name_list(file_in):
    
    name_list = []
    with open(file_in, 'r') as f_in:
        line = f_in.readline()
        line = f_in.readline()
        while line:          
            name_list.append(line[2:].strip())
            line = f_in.readline()

    return name_list

rxn_names = get_name_list(file_in_rxns)
enz_names = get_name_list(file_in_enzs)
met_names = get_name_list(file_in_mets)

def get_df_median(mat, key_name, id_col, col_names, row_names, n_models):
    
    try:
        df = pd.DataFrame(data=mat['mcaResults'][key_name].item())
        df.columns = col_names
    except ValueError:
        df = pd.DataFrame(data=mat['mcaResults'][key_name].item()[0])
        df.columns = col_names

    df[id_col] = np.tile(row_names, n_models)
    df_median = df.groupby(id_col).median()
    
    return df_median


def get_df_iqr(mat, key_name, id_col, col_names, row_names, n_models):
    
    try:
        df = pd.DataFrame(data=mat['mcaResults'][key_name].item())
        df.columns = col_names
    except ValueError:
        df = pd.DataFrame(data=mat['mcaResults'][key_name].item()[0])
        df.columns = col_names

    df[id_col] = np.tile(row_names, n_models)
   
    df_q2 = df.groupby(id_col).quantile(0.25)
    df_q4 = df.groupby(id_col).quantile(0.75)
    
    df_iqr = df_q4.subtract(df_q2)    

    return df_iqr

In [6]:
mat = scipy.io.loadmat(file_in_MCA, squeeze_me=True)

In [7]:
#change to that for concentration control coefficients
key_name = 'xControl'
id_col = 'mets'

In [8]:
col_names = rxn_names
key_name = 'xControl'
df = pd.DataFrame(data=mat['mcaResults'][key_name].item())
df.columns = col_names
id_col = 'mets'
n_models = 10000
row_names = met_names
df[id_col] = np.tile(row_names, n_models)

In [10]:
df.head()

R_GLCabcpp     R_GLK  R_GLCNt2rpp     R_GNK  R_2DHGLCNkt_tpp  R_2DHGLCK  \
0   -0.306002 -0.059338    -0.226028 -0.040838        -0.015915  -0.044226   
1    0.978987 -0.711211    -0.324106 -0.407581        -0.022482  -0.118067   
2    3.047573 -3.078397     0.776585  0.760549         0.087963   0.367772   
3    1.140592 -0.747165    -0.407531 -0.333786        -0.047873  -0.189994   
4    0.407369  0.445886     0.175182  0.138557         0.021691   0.082030   

   R_PGLCNDH_NAD  R_PGLCNDH_NADP  R_GLCDpp  R_GAD2ktpp  ...  R_EX_akg  \
0      -0.002668       -0.021873 -3.609335    0.054071  ...  0.031507   
1      -0.001041       -0.003285  0.090837    0.062705  ... -0.076163   
2       0.008351        0.063940 -1.512878   -0.339045  ...  0.093639   
3      -0.005582       -0.045913  0.101079    0.192912  ... -0.000800   
4       0.001670        0.011499  0.017075   -0.082419  ...  0.045498   

   R_EX_succ  R_EX_ru5p__D  R_EX_fum  R_EX_s7p  R_EX_13dpg  R_EX_dhap  \
0  -0.000484     -0.000430 -0.002833 -0.000765    0.003969  -0.001266   
1   0.001359     -0.000449  0.007335 -0.002279    0.010131  -0.003822   
2  -0.001442      0.000777 -0.000069  0.000828   -0.018939   0.006260   
3   0.000396     -0.001280 -0.007030 -0.003611    0.011246  -0.003942   
4  -0.000029     -0.001092 -0.004864 -0.005393   -0.000394  -0.000498   

   R_EX_amp_in  R_EX_amp_out        mets  
0          0.0           0.0  m_glc__D_p  
1          0.0           0.0     m_atp_c  
2          0.0           0.0  m_glc__D_c  
3          0.0           0.0     m_adp_c  
4          0.0           0.0     m_g6p_c  

[5 rows x 79 columns]

In [12]:
id_col = 'mets'
df_melted = df.melt(id_vars=[id_col], var_name='variable')

In [14]:
#change "__" for "-"
#change "m_" for ""
#change "R_" for ""
#change "_" for " "

In [ ]:
# It will take a lot of time to run. If unnecessary, skip this line. Further on change accoa c to accoa_c, and add R_ to the beginning of the reaction name
df_melted.mets = df_melted.mets.str.replace('__', '-', regex=True).replace('m_', '', regex=True).replace('_c', '').replace('_', ' ', regex=True)

In [ ]:
df_melted.variable = df_melted.variable.str.replace('R_', '', regex=True).replace('_', ' ', regex=True)

In [ ]:
a = df_melted.groupby(['mets','variable']).apply(lambda x : \
          x[(x.value<x.value.quantile(0.8)) & (x.value>x.value.quantile(0.2))])
a.index = range(1,a.shape[0]+1)

In [24]:
import matplotlib.colors as mpc

In [ ]:
plot_data = a[a.variable== 'CS'].copy() 
plot_data = plot_data[plot_data.variable != 'MDH NAD'] # for graph readability
plot_data = plot_data[plot_data.variable != 'ME2 NAD'] # for graph readability
plot_data["median_value"] = plot_data.groupby("mets")["value"].transform("median")
##Trying to create a colormap 
norm = plt.Normalize(plot_data["median_value"].min(), plot_data["median_value"].max())
ar = plot_data.groupby("mets")["value"].median().values

original_cmap = plt.cm.RdBu_r
#cmap = mpc.ListedColormap(original_cmap(norm(ar)))

divnorm = mpc.DivergingNorm(vmin=-1., vcenter=0, vmax=1)
cmap = mpc.ListedColormap(original_cmap(divnorm(ar)))

In [ ]:
fig, axes = joypy.joyplot(
    plot_data,
    by="mets",
    column="value",
    figsize=(6,10),
    ylim='own',
    linewidth=1,
    colormap = cmap,
    title="concentration control coefficient of CS")
plt.savefig('../..figures/CS_CCC_ridge.svg')

In [ ]:
plot_data = a[a.variable== 'ACCOAC'].copy() 
plot_data = plot_data[plot_data.variable != 'MDH NAD']
plot_data = plot_data[plot_data.variable != 'ME2 NAD']
plot_data["median_value"] = plot_data.groupby("mets")["value"].transform("median")
##Trying to create a colormap 
norm = plt.Normalize(plot_data["median_value"].min(), plot_data["median_value"].max())
ar = plot_data.groupby("mets")["value"].median().values

original_cmap = plt.cm.RdBu_r
#cmap = mpc.ListedColormap(original_cmap(norm(ar)))

divnorm = mpc.DivergingNorm(vmin=-1., vcenter=0, vmax=1)
cmap = mpc.ListedColormap(original_cmap(divnorm(ar)))

In [ ]:
fig, axes = joypy.joyplot(
    plot_data,
    by="mets",
    column="value",
    figsize=(6,10),
    ylim='own',
    linewidth=1,
    colormap = cmap,
    title="concentration control coefficient of ACCOAC")
plt.savefig('../..figures/ACCOAC_CCC_ridge.svg')

In [ ]:
plot_data = a[a.mets== 'accoa c'].copy() 
plot_data = plot_data[plot_data.variable != 'MDH NAD']
plot_data = plot_data[plot_data.variable != 'ME2 NAD']
plot_data["median_value"] = plot_data.groupby("variable")["value"].transform("median")
##Trying to create a colormap 
norm = plt.Normalize(plot_data["median_value"].min(), plot_data["median_value"].max())
ar = plot_data.groupby("variable")["value"].median().values

original_cmap = plt.cm.RdBu_r
#cmap = mpc.ListedColormap(original_cmap(norm(ar)))

divnorm = mpc.DivergingNorm(vmin=-1., vcenter=0, vmax=1)
cmap = mpc.ListedColormap(original_cmap(divnorm(ar)))

In [ ]:
fig, axes = joypy.joyplot(
    plot_data,
    by="variable",
    column="value",
    figsize=(6,13),
    ylim='own',
    linewidth=1,
    colormap = cmap,
    title="AcCoA concentration control coefficient")
plt.savefig('../..figures/ACCOA_CCC_ridge.svg')